In [1]:
import numpy as np
from astroquery.gaia import Gaia
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.io import ascii
Gaia.ROW_LIMIT = 2000

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
# Gaia J2015.5 coordinates
coord_file = '/Users/mguzzetti/Desktop/githubrepo/MinMs/WideCompanionSearch/new_MinMs_coordinates.txt'
# corresponding HIP names
name_file = '/Users/mguzzetti/Desktop/githubrepo/MinMs/WideCompanionSearch/MinMs_names.txt'
# Gaia parallaxes to calculate search radius
plx_file ='/Users/mguzzetti/Desktop/githubrepo/MinMs/WideCompanionSearch/MinMs_plx.txt'
# read in files
ra, dec  = np.loadtxt(coord_file, unpack = True)
names  = np.loadtxt(name_file, unpack = True)
plx = np.loadtxt(plx_file, unpack = True)


In [4]:
##### N SIGMA CUT-OFF ######N = 50
# initialize candidate table 
cand_table = Table(names=('HIP_num','sep_arcsec', 'sep_AU','ra', 'ra_err', 'dec','dec_error','parallax', 'parallax_error','pmra','pmra_error','pmdec',
                         'pmdec_error','bp_mag','g_mag','rp_mag','teff'))
# loop through MinMs target list
for i in range(len(ra)):
    print('Searching HIP', int(names[i]))
    # read coordinate of target i out of 234 (all 245 weren't in Gaia)
    coord = SkyCoord(ra=ra[i], dec=dec[i], unit=(u.degree, u.degree), frame='icrs')
    # calculate angular search radius scaled for distance (10,000 AU projected sep)
    rad_val = plx[i]/360
    radius = u.Quantity(rad_val, u.deg)
    # query all Gaia sources within the search radius
    r = Gaia.query_object_async(coordinate=coord, radius=radius)
    # loop through all of the sources around the target to search for common proper motion
    for j in range(len(r)):
        # don't include the first object (aka, the target itself)
        if j == 0:
            pass
        else:
            # array of target proper motions and parallax
            targ = np.array((r['pmra'][0],r['pmdec'][0],r['parallax'][0])) # target
            # array of object j proper motions and parallax
            s = np.array((r['pmra'][j],r['pmdec'][j],r['parallax'][j]))  
            diff = np.abs(targ-s) #difference between the two in the 3 parameters  
            # check if the difference in the three components is less than N sigma 
            if diff[0] < N*np.asarray(r['pmra_error'][0]) and diff[1] < N*np.asarray(r['pmdec_error'][0]) and diff[2] <  N*np.asarray(r['parallax_error'][0]):
                # coordinates of potential companion
                c1 = SkyCoord(ra=r['ra'][j], dec=r['dec'][j], unit=(u.degree, u.degree),frame='icrs')
                # coordinates of target of interest
                c2 = SkyCoord(ra=ra[i], dec=dec[i], unit=(u.degree, u.degree),frame='icrs')
                # calculate separation between the two and convert to AU
                sep = c1.separation(c2)
                arcsec = (sep*3600).value
                dist = 1000/plx[i]
                AU = dist*arcsec
                # write candidate info into table
                cand_table.add_row((int(names[i]),arcsec, AU, r['ra'][j],r['ra_error'][j],r['dec'][j],r['dec_error'][j],r['parallax'][j],r['parallax_error'][j],r['pmra'][j],
                   r['pmra_error'][j],r['pmdec'][j],r['pmdec_error'][j],r['phot_bp_mean_mag'][j],r['phot_g_mean_mag'][j],
                   r['phot_rp_mean_mag'][j],r['teff_val'][j]))
                print("CANDIDATE FOUND")           
ascii.write(cand_table, 'Nsigma_candidates.txt', format='commented_header',overwrite=True)
    

Searching HIP 428
INFO: Query finished. [astroquery.utils.tap.core]


/Users/mguzzetti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Warning: converting a masked element to nan.


Searching HIP 439
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 1475
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 2552
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 3937
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 4569
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 4856
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 4872
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 5496
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 5643
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 8051
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 8768
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 9291
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 9724
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 9786
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 10279
INFO: Query fin

Searching HIP 61706
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 61874
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 62452
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 63510
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 65011
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 65026
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 65714
INFO: Query finished. [astroquery.utils.tap.core]
CANDIDATE FOUND
Searching HIP 65859
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 66625
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 66906
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 67155
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 67164
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 68469
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 69454
INFO: Query finished. [astroquery.utils.tap.core]
Sear

In [5]:
##### FRACTIONAL CUT-OFF ######
frac = 0.2 
# initialize candidate table 
cand_table = Table(names=('HIP_num','sep_arcsec', 'sep_AU','ra', 'ra_err', 'dec','dec_error','parallax', 'parallax_error','pmra','pmra_error','pmdec',
                         'pmdec_error','bp_mag','g_mag','rp_mag','teff'))
# loop through MinMs target list
for i in range(len(ra)):
    print('Searching HIP', int(names[i]))
    # read coordinate of target i out of 234 (all 245 weren't in Gaia)
    coord = SkyCoord(ra=ra[i], dec=dec[i], unit=(u.degree, u.degree), frame='icrs')
    # calculate angular search radius scaled for distance (10,000 AU projected sep)
    rad_val = plx[i]/360
    radius = u.Quantity(rad_val, u.deg)
    # query all Gaia sources within the search radius
    r = Gaia.query_object_async(coordinate=coord, radius=radius)
    # loop through all of the sources around the target to search for common proper motion
    for j in range(len(r)):
        # don't include the first object (aka, the target itself)
        if j == 0:
            pass
        else:
            # array of target proper motions and parallax
            targ = np.array((r['pmra'][0],r['pmdec'][0],r['parallax'][0])) # target
            # array of object j proper motions and parallax
            s = np.array((r['pmra'][j],r['pmdec'][j],r['parallax'][j]))  
            diff = np.abs(targ-s) #difference between the two in the 3 parameters 
            # check if the difference in the three components is less than (frac*100)% of the primary's values
            if diff[0] < np.abs(frac*targ[0]) and diff[1] < np.abs(frac*targ[1]) and diff[2] < frac*targ[2]:
                # coordinates of potential companion
                c1 = SkyCoord(ra=r['ra'][j], dec=r['dec'][j], unit=(u.degree, u.degree),frame='icrs')
                # coordinates of target of interest
                c2 = SkyCoord(ra=ra[i], dec=dec[i], unit=(u.degree, u.degree),frame='icrs')
                # calculate separation between the two and convert to AU
                sep = c1.separation(c2)
                arcsec = (sep*3600).value
                dist = 1000/plx[i]
                AU = dist*arcsec
                # write candidate info into table
                cand_table.add_row((int(names[i]),arcsec, AU, r['ra'][j],r['ra_error'][j],r['dec'][j],r['dec_error'][j],r['parallax'][j],r['parallax_error'][j],r['pmra'][j],
                   r['pmra_error'][j],r['pmdec'][j],r['pmdec_error'][j],r['phot_bp_mean_mag'][j],r['phot_g_mean_mag'][j],
                   r['phot_rp_mean_mag'][j],r['teff_val'][j]))
                print("CANDIDATE FOUND")           
ascii.write(cand_table, 'frac_candidates.txt', format='commented_header',overwrite=True)

Searching HIP 428
INFO: Query finished. [astroquery.utils.tap.core]
CANDIDATE FOUND
CANDIDATE FOUND


/Users/mguzzetti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Warning: converting a masked element to nan.


Searching HIP 439
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 1475
INFO: Query finished. [astroquery.utils.tap.core]
CANDIDATE FOUND
Searching HIP 2552
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 3937
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 4569
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 4856
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 4872
INFO: Query finished. [astroquery.utils.tap.core]
CANDIDATE FOUND
Searching HIP 5496
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 5643
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 8051
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 8768
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 9291
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 9724
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 9786
INFO: Query finished. [astroquery.utils.tap.core]
Sea

INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 60910
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 61094
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 61629
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 61706
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 61874
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 62452
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 63510
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 65011
INFO: Query finished. [astroquery.utils.tap.core]
CANDIDATE FOUND
Searching HIP 65026
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 65714
INFO: Query finished. [astroquery.utils.tap.core]
CANDIDATE FOUND
Searching HIP 65859
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 66625
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 66906
INFO: Query finished. [astroquery.utils.tap.core]
Searchin

Searching HIP 114046
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 115332
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 115562
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 116132
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 116317
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 117473
INFO: Query finished. [astroquery.utils.tap.core]
Searching HIP 117828
INFO: Query finished. [astroquery.utils.tap.core]
